# Importando Pacotes


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as wg
from scipy import signal as sg
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve

#!pip install bees-algorithm;
#from bees_algorithm import BeesAlgorithm


from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.io as pio
pio.templates.default = 'plotly_dark' #seaborn' #'seaborn'
Opcoes = ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none'];

# Configurando para os gráficos serem inseridos no Notebook
%matplotlib inline

## Malha fechada com controladores para h e T4

In [2]:
def PID_p2(SP, PV,k,I_int,D_int, dt, Method = 'Backward', Kp=10.0, Ti=50.0, Td=1.0, b=1., c=0.0, 
           N=10., U_bias = 0., Umin = -100., Umax = 100.):
    # PID -- posicional implementation   
    # Para sistemas não lineares com ação integral e b!=1 inicializar I_int =  Kp*Yset_bias*(1-b)    
    # The approximation of the derivative term are stable only when abs(ad)<1. 
    # The forward difference approximation requires that Td>N*dt/2, i.e., the 
    # approximation becomes unstable for small values of Td. The other approximation
    # are stable for all values of Td. 
    
    # Autor: Jorge Otávio Trierweiler -- jorge.trierweiler@ufrgs.br
    
    if Method == 'Backward':
        b1 = Kp*dt/Ti if Ti != 0 else 0.0
        b2 = 0.0
        ad = Td/(Td+N*dt)
        bd = Kp*Td*N/(Td+N*dt)
    elif Method == 'Forward':
        b1 = 0.0
        b2 = Kp*dt/Ti  if Ti!=0 else 0.0
        ad = 1-N*dt/Td if Td!=0 else 0.0
        bd = Kp*N   
    elif Method == 'Tustin':
        b1 = Kp*dt/2/Ti if Ti!=0 else 0.0
        b2 = b1
        ad = (2*Td-N*dt)/(2*Td+N*dt)
        bd = 2*Kp*Td*N/(2*Td+N*dt)   
    elif Method == 'Ramp':
        b1 = Kp*dt/2/Ti if Ti!=0 else 0.0
        b2 = b1
        ad = np.exp(-N*dt/Td) if Td !=0 else 0.0
        bd = Kp*Td*(1-ad)/dt
        
    # Derivative Action 
    D  = ad*D_int+bd*((c*SP[k]-PV[k])-(c*SP[k-1]-PV[k-1]))
    
    # Integral action
    II = b1*(SP[k]-PV[k])+b2*(SP[k-1]-PV[k-1])
    #II = Kp*dt/Ti*(SP[k]-PV[k]) if Ti!=0 else 0.0
    # print(I_int, II)
    I = I_int + II                         
   
    # calculate the PID output
    P = Kp * (b*SP[k]-PV[k])
       
    # output
    Uop = U_bias + P + I + D
    #print(P, I, D, Uop)

    # implement anti-reset windup
    if Uop < Umin:
        II = 0.0      # no caso de saturação -- diminuindo o valor integrado a mais
        Uop = Umin
    if Uop > Umax:
        II = 0.0
        Uop = Umax
        
    # return the controller output and PID terms
    return np.array([Uop, I_int+II, D])

In [3]:
# T0 = h, T3, Tq, T4a

In [4]:
# modelo do chuveiro turbinado
def TanqueTub(t, z,                                   # tempo e variável de estado -- seguindo o padrão do integrado
              Sr=100, Sa=100, xq=0.3, xf=0.3, xs=0.4,  # manipuladas
              Fd=0, Td=25, Tinf=25):                  # distúrbios -- entradas
    
    #print(Sr, Sa, xq, xf, xs, Fd, Td, Tinf)
    
    # variáveis de estado
    h, T3, Tq, T4a = z
    
    #Parâmetros do tanque
    A = 0.5

    #Definindo Tf
    Tf = Td

    # implementação das vazões Fq, Ff e Fs
    t1 = xq**0.2; t2 = t1**2; t5 = xf ** 2; t7 = xq ** 2; t9 = t1 * xq * t7; t10 = t9 * t5;
    t14 = t7 ** 2; t16 = t2 * t7 * t14; t17 = t16 * t5; t20 = np.sqrt(0.9e1 * t10 + 0.50e2 * t17); t26 = t5 ** 2;
    t38 = (np.sqrt(-0.1e1 * (-0.180e3 -0.45e2 * t5 -0.250e3 * t10 - 0.1180e4 * t9 + 0.60e2 * t20) / (0.6552e4 * t10 + 0.648e3 
           * t5 + 0.16400e5* t17 + 0.900e3 * t9 * t26 + 0.2500e4 * t16 * t26 + 0.81e2 * t26 + 0.55696e5 * t16 + 0.16992e5 
           * t9 + 0.1296e4)));
    Fq = 60 * t1 * t2 * xq * t38;
    
    Ff = (2*xf*np.sqrt(125*xf**2 - Fq**2 + 500) - xf*Fq) / (xf**2 + 4)
    
    Fs = (5*xs**3*np.sqrt(30)*np.sqrt(-15*xs**6 + np.sqrt(6625*xs**12 + 640*xs**6 + 16))
          / (20*xs**6 + 1))
    
    #equações diferenciais
    return([(Fq + Ff + Fd - Fs)/A, 
            (Ff*(Tf - T3) + Fq*(Tq - T3) + Fd*(Td - T3) + Sr*80/100)/(A*h),
            Fq*(Tf - Tq) + 50*Sa/100, 
            Fs/2.5*(T3 - T4a) - 0.8*((T3 -Tinf)*(T4a - Tinf)*(1/2*(T3 - Tinf) + 1/2*(T4a - Tinf)))**(1/3)])

In [30]:
# controlador liga-desliga
def ON_OFF (ONOFF, y_medido, y_sp, banda_morta=1):
    ONOFF_actual = ONOFF
    if y_medido < (y_sp - banda_morta):
        ONOFF_actual = 100
        return ONOFF_actual
    elif y_medido > (y_sp + banda_morta):
        ONOFF_actual = 0.0
        return ONOFF_actual
    else:
        return ONOFF_actual

In [94]:
def Simulacao_MF_Final(SYS, y0, TU, 
                       Kp_T4a_ext=20, Kp_T4a_int=1, 
                       Ti_T4a_ext=1, Ti_T4a_int=0.1, 
                       Td_T4a_ext=0.0, Td_T4a_int=0.0, 
                       b_T4a_ext=1, b_T4a_int=1,
                       Kp_h=2, Ti_h=0.5, Td_h=0.0, b_h=1,
                       Ruido=0.005, U_bias_T4a=50, U_bias_h=0.5,
                       dt=0.01, Matplot=False, Plotly=False):
    
    Kp_T4a = [Kp_T4a_ext, Kp_T4a_int]
    Ti_T4a = [Ti_T4a_ext, Ti_T4a_int]
    Td_T4a = [Td_T4a_ext, Td_T4a_int]
    b_T4a = [b_T4a_ext, b_T4a_int]
    
    # Definição tempo final e condições iniciais da simulação
    Tfinal = TU[-1,0] 
    
    Yset_bias_T4a = [y0[-1], y0[1]]
    Yset_bias_h = y0[0]

    # Armazenamento dos dados resultantes da simulação  
    TT = np.arange(start=0, stop=Tfinal+dt, step=dt, dtype='float')
    NT = np.size(TT)
    NY = np.size(y0)
    YY = np.zeros((NT,NY))
    nu = np.size(TU,1)
    UU = np.zeros((NT,nu-1))
    SP0_T4a = np.zeros_like(TT)
    SP1_T4a = np.zeros_like(TT)
    PV0_T4a = np.ones_like(TT)*Yset_bias_T4a[0]
    PV1_T4a = np.ones_like(TT)*Yset_bias_T4a[1]
    SP_h = np.zeros_like(TT)
    PV_h = np.ones_like(TT)*Yset_bias_h
    SP_Tq = np.zeros_like(TT)
    Noise = np.random.normal(0, Ruido, len(TT))
    
    #YY[-1,2] = y0[2]

    ii = 0    
    
    D_int0_T4a = 0.0  
    I_int0_T4a = Kp_T4a[0]*Yset_bias_T4a[0]*(1-b_T4a[0])
    
    D_int1_T4a = 0.0  
    I_int1_T4a = Kp_T4a[1]*Yset_bias_T4a[1]*(1-b_T4a[1])
    
    D_int_h = 0.0  
    I_int_h = Kp_h*Yset_bias_h*(1-b_h)

    YY[0,:] = y0
    
    onoff = 100

    for k in np.arange(NT-1):
        
        if TT[k] >= TU[ii+1,0]:
            ii=ii+1
    
        # Definição do setpoint e do distúrbio na carga
        UU[k,:] = TU[ii,1:nu]
        #print(UU[k,:])
    
        # SP0_T4a[k] = TU[ii,1]
        SP0_T4a[k] = TU[ii,3]
         
        PV0_T4a[k] = YY[k,-1] + Noise[k]  
        PV1_T4a[k] = YY[k,1] + Noise[k]
        
        SP_h[k] = TU[ii,4]
        PV_h[k] = YY[k,0] + Noise[k]
     
        SP_Tq[k] = TU[ii,2]

        # Armazenamento dos valores calculados
        # Vazão de saída:
        SP_Fs = TU[ii,5]
        def compute_out_flow(xs):
            return (5*xs**3*np.sqrt(30)*np.sqrt(-15*xs**6 + np.sqrt(6625*xs**12 + 640*xs**6 + 16)) / (20*xs**6 + 1)) - SP_Fs
        xs_solve = fsolve(compute_out_flow, 0.5)
        
        #Malha nível
        #print('Malha nível')
        uu_h = PID_p2(SP_h,PV_h,k,I_int_h, D_int_h, dt, Method='Backward',
                Kp=Kp_h,Ti=Ti_h,Td=Td_h,N=10,b=b_h, 
                Umin=0, Umax=1, U_bias=U_bias_h)
        Uop_h   = uu_h[0]
        I_int_h = uu_h[1]
        D_int_h = uu_h[2]

        # Malhas T4a
        # Malha externa (C1)
        #print('Malha externa')
        #print(SP0_T4a[k], PV0_T4a[k])
        uu = PID_p2(SP0_T4a,PV0_T4a,k,I_int0_T4a, D_int0_T4a, dt, Method='Backward',
                Kp=Kp_T4a[0],Ti=Ti_T4a[0],Td=Td_T4a[0],N=10,b=b_T4a[0], 
                Umin=20, Umax=60, U_bias=Yset_bias_T4a[1])
        SP1_T4a[k] = uu[0]
        I_int0_T4a = uu[1]
        D_int0_T4a = uu[2]
        
        # Malha interna (C2)
        #print('Malha interna')
        #print(SP1_T4a[k], PV1_T4a[k])
        uu = PID_p2(SP1_T4a,PV1_T4a,k,I_int1_T4a, D_int1_T4a, dt, Method='Backward',
                Kp=Kp_T4a[1],Ti=Ti_T4a[1],Td=Td_T4a[1],N=10,b=b_T4a[1], 
                Umin=0.05, Umax=0.3, U_bias=U_bias_T4a)
        Uop_T4a    = uu[0]
        #print(Uop_T4a)
        I_int1_T4a = uu[1]
        D_int1_T4a = uu[2]
        
        # Malha Boiler
        Uop_Tq = ON_OFF(onoff, YY[k,2], SP_Tq[k], banda_morta=1)
        
        # Definição do setpoint e do distúrbio na carga
        # UU[k,0] = Uop_T4a
        UU[k, 2] = Uop_T4a
        UU[k, 3] = Uop_h
        UU[k, 1] = Uop_Tq
        UU[k, 4] = xs_solve
        
        sol = solve_ivp(SYS, [TT[k], TT[k+1]], YY[k,:], args=tuple(UU[k,:]), rtol=1e-6) #,method="RK45",max_step = dt, atol=1, rtol=1)
                        
        YY[k+1,:] = sol.y[:,-1]
                        
    #erro = yy_f-YY
    
    UU[k+1,:] = TU[ii,1:nu]
    # UU[k+1,0] = Uop_T4a
    UU[k+1,2] = Uop_T4a
    # SP0_T4a[k+1] = TU[ii,1]
    SP0_T4a[k+1] = TU[ii,3]
    SP1_T4a[k+1] = SP1_T4a[k]

    UU[k+1,3] = Uop_h
    SP_h[k+1] = TU[ii,4]
    
    UU[k+1,4] = xs_solve
    
    UU[k+1,1] = Uop_Tq
  
    if Matplot:
        plt.figure(figsize=(12,6))
        plt.subplot(2,1,1)    
        
        plt.plot(TT,YY[:,0])            # respota obtida em simulação discreta
        plt.plot(TT,YY[:,-1]) 
        plt.plot(TT,SP0_T4a)
        plt.plot(TT,SP1_T4a)
        plt.title('Saídas -- ($T_1$. $T_f$, $SP_1$ e $SP_2$)')
        plt.legend(['T1','Tf','$SP_f$','$SP_1$'])
        #plt.plot(r['time'],'y']) # resposta anterior -- rodando tudo de uma só vez
        plt.grid()
        plt.subplot(2,1,2)
        plt.plot(TT,UU)
        plt.title('Entradas  (MV e distúrbios)')
        plt.xlabel('Tempo [min]')
        plt.legend(['z','Fin','Tin','Tinf'])
        plt.grid()
        
    if Plotly:
        fig = make_subplots(4,1)
        fig.add_trace(go.Scatter(name='T4a', x=TT, y=YY[:,-1]),1,1)
        fig.add_trace(go.Scatter(name='T3', x=TT, y=YY[:,1]),1,1)
        fig.add_trace(go.Scatter(name='SP(T4a)', x=TT, y=SP0_T4a,line= {"shape": 'hv'}),1,1)
        fig.add_trace(go.Scatter(name='Tq', x=TT, y=YY[:,2],line= {"shape": 'hv'}),1,1)
          
        fig.add_trace(go.Scatter(name='h', x=TT, y=YY[:,0]),2,1)
        fig.add_trace(go.Scatter(name='SP(h)', x=TT, y=SP_h,line= {"shape": 'hv'}),2,1)
        
        fig.add_trace(go.Scatter(name='Sr', x=TT, y=UU[:,0],line= {"shape": 'hv'}),3, 1 )        
        fig.add_trace(go.Scatter(name='Sa', x=TT, y=UU[:,1],line= {"shape": 'hv'}),3, 1 ) 
        
        fig.add_trace(go.Scatter(name='xq', x=TT, y=UU[:,2],line= {"shape": 'hv'}),4, 1 ) 
        fig.add_trace(go.Scatter(name='xf', x=TT, y=UU[:,3],line= {"shape": 'hv'}),4, 1 ) 
        fig.add_trace(go.Scatter(name='xs', x=TT, y=UU[:,4],line= {"shape": 'hv'}),4, 1 ) 
        
        #fig.add_trace(go.Scatter(name='Fd', x=TT, y=UU[:,5],line= {"shape": 'hv'}),5, 1 ) 
        #fig.add_trace(go.Scatter(name='Td', x=TT, y=UU[:,6],line= {"shape": 'hv'}),5, 1 ) 
        #fig.add_trace(go.Scatter(name='Tinf', x=TT, y=UU[:,7],line= {"shape": 'hv'}),5, 1 ) 
        fig.update_xaxes(matches='x')
  
        
        fig.update_layout(
            height=900, 
            width=800,
            title={
            #'text': 'Saídas',
            'y':0.85,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            #xaxis_title = r'Entradas (MV e Distúrbios)',
            #yaxis_title='Temperaturas',
            legend_title="Variáveis",
        )
        fig.show() 

        
    return (TT,YY,UU)

In [134]:
TU =np.array(
[    #Time,      Sr, SP(Tq), SP(T4a), SP(h), SP(Fs),  Fd,  Td,  Tinf
      [  0,       80,   40,   37,     60,      5,     0,  25,   25],
      [ 10,       80,   40,   37,     60,      5,     0,  25,   25],
      [ 20,       80,   40,   37,     60,      5,     0,  25,   25],
      [ 30,       80,   40,   37,     60,      5,     0,  25,   25],
      [ 40,       80,   42,   37,     60,      5,     0,  25,   25],
      [ 50,       80,   42,   37,     60,      5,     0,  25,   25],
      [ 60,       80,   42,   37,     60,      5,     0,  25,   25],
      [ 70,       80,   42,   37,     60,      5,     0,  25,   25],
      [ 80,       80,   40,   37,     60,      5,     0,  25,   25],
      [ 90,       80,   40,   37,     60,      5,     0,  25,   25],
      [100,       80,   40,   37,     60,      5,     0,  25,   25],
])

TU =np.array(
[    #Time,      Sr, SP(Tq), SP(T4a), SP(h), SP(Fs),  Fd,  Td,  Tinf
      [  0,       70,   42,   37,     60,      5,     0,  25,   25],
      [ 10,       50,   38,   34,     60,      5,     0,  25,   25],
      [ 20,       50,   38,   34,     60,      5,     0,  25,   25],
])

TU =np.array(
[    #Time,      Sr, SP(Tq), SP(T4a), SP(h), SP(Fs),  Fd,  Td,  Tinf
      [  0,       70,   42,   37,     60,      5,     0,  25,   25],
      [ 10,       70,   45,   38,     60,      5,     0,  25,   25],
      [ 20,       70,   45,   38,     60,      5,     0,  25,   25],
])

T0 = [60,  35,  42,  35]

#Simulacao_MF_Final(TanqueTub, T0, TU, Kp_T4a=[20.63,1], Ti_T4a=[1.53,1e6], Td_T4a=[0.0,0.0], b_T4a=[1,1],
#                     Kp_h= 1, Ti_h=0.3, Td_h=0.0, b_h=1, 
#                     Ruido = 0.005, U_bias_T4a=50, U_bias_h=0.5, dt=0.01, Plotly = True );

# Kp: 1.3 e 1.0
# Ti: 2.4 e 1.1

Simulacao_MF_Final(TanqueTub, T0, TU, 
                   Kp_T4a_ext=1.16, Kp_T4a_int=1.06,  # Kp_T4a_int=0.21, Kp_T4a_ext=1.61
                   Ti_T4a_ext=2.35, Ti_T4a_int=1.25, # Ti_T4a_int=0.3, Ti_T4a_ext=1.2,
                   Td_T4a_ext=0.0, Td_T4a_int=0.0, 
                   b_T4a_ext=1, b_T4a_int=1,
                   Kp_h=1, Ti_h=0.3, Td_h=0.0, b_h=1,
                   Ruido=0.005, U_bias_T4a=0.0, U_bias_h=0.5,
                   dt=0.01, Plotly=True);

In [135]:
def wg_MF(Kp_T4a_int=1.06, Kp_T4a_ext=1.16, 
          Ti_T4a_int=1.25, Ti_T4a_ext=2.35,
          Td_T4a_int=0.0, Td_T4a_ext=0.0,
          b_T4a_int=1.0, b_T4a_ext=1.0):
          #Kp_h=1, Ti_h=0.3, Td_h=0.0, b_h=1):
    Simulacao_MF_Final(TanqueTub, T0, TU, 
                       Kp_T4a_int=Kp_T4a_int, Kp_T4a_ext=Kp_T4a_ext, 
                       Ti_T4a_int=Ti_T4a_int, Ti_T4a_ext=Ti_T4a_ext, 
                       Td_T4a_int=Td_T4a_int, Td_T4a_ext=Td_T4a_ext, 
                       b_T4a_int=b_T4a_int, b_T4a_ext=b_T4a_ext,
                       Kp_h=1, Ti_h=0.3, Td_h=0.0, b_h=1, 
                       Ruido=0.005, U_bias_T4a=0.5, U_bias_h=0.5,
                       dt=0.01, Plotly=True);

In [136]:
wg.interact(wg_MF,
            Kp_T4a_int=(0.01, 5, 0.05), Kp_T4a_ext=(0.01, 5, 0.05), 
            Ti_T4a_int=(0.0, 5, 0.05), Ti_T4a_ext=(0.0, 5, 0.05),
            Td_T4a_int=(0.0, 3), Td_T4a_ext=(0.0, 3), 
            b_T4a_int=(0.01, 4), b_T4a_ext=(0.01, 4))
            #Kp_h=(0.0, 10), Ti_h=(0.0, 5), Td_h=(0.0, 1), b_h=(0.0, 2))

interactive(children=(FloatSlider(value=1.06, description='Kp_T4a_int', max=5.0, min=0.01, step=0.05), FloatSl…

<function __main__.wg_MF(Kp_T4a_int=1.06, Kp_T4a_ext=1.16, Ti_T4a_int=1.25, Ti_T4a_ext=2.35, Td_T4a_int=0.0, Td_T4a_ext=0.0, b_T4a_int=1.0, b_T4a_ext=1.0)>

In [137]:
# Kps int e ext = 12.61 e 0.71 -> xq de 0 a 3
# Tis int e ext = 2.5 e 0.4

# Kps int e ext = 22.81 e 2.31 -> xq de 0.05 a 3
# Tis int e ext = 3.8 e 0.9

In [138]:
# Custo gás - contar quanto tempo Sa fica ligada e desligada e depois fazer regra de 3 do custo total
# https://www.vivaceinstruments.com.br/pt/artigo/o-que-e-um-controle-split-range-com-posicionadores-de-valvula